In [22]:
import ee
import geemap

# Autentikasi dan inisialisasi Earth Engine
ee.Authenticate()
ee.Initialize(project='map-mappings-409708')

def cloudMask(image):
    qa = image.select('QA_PIXEL')
    dilated = 1 << 1
    cirrus = 1 << 2
    cloud = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(dilated).eq(0) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return image.select(['SR_B.*'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']).multiply(0.0000275).add(-0.2).updateMask(mask)

admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
roi1 = admin2.filter(ee.Filter.eq('ADM1_NAME', 'Bali'))
geometry = roi1.geometry()
Map = geemap.Map()
Map.centerObject(geometry,zoom=8)
Map.addLayer(geometry, {'color': 'grey'}, 'Prov. Bali') 

# Get water data
gsw = ee.Image("JRC/GSW1_1/GlobalSurfaceWater")
water = gsw.select('occurrence').clip(roi1)
Map.addLayer(water, {'min': 0, 'max': 100, 'palette': ['white', 'cyan', 'blue']}, 'Water', False) 

# Permanent watera
permanent = water.gt(80)
Map.addLayer(permanent.selfMask(), {'palette': 'blue'}, 'Permanent Water', False) 

# Rainbow palette
rainbow = ['blue', 'cyan', 'green', 'yellow', 'red']

# Distance from water
distance = permanent.fastDistanceTransform().divide(30).clip(roi1).reproject('EPSG:4326', None, 30)
Map.addLayer(distance, {'max': 0, 'min': 5000, 'palette': rainbow}, 'Distance', False) 

# Only the distance without permanent water
onlyDistance = distance.updateMask(distance.neq(0))
Map.addLayer(onlyDistance, {'min': 0, 'max': 5000, 'palette': rainbow}, 'Distance from permanent water', False) 

# Distance
distanceScore = onlyDistance.where(onlyDistance.gt(4000), 1) \
    .where(onlyDistance.gt(3000).And(onlyDistance.lte(4000)), 2) \
    .where(onlyDistance.gt(2000).And(onlyDistance.lte(3000)), 3) \
    .where(onlyDistance.gt(1000).And(onlyDistance.lte(2000)), 4) \
    .where(onlyDistance.lte(1000), 5)
Map.addLayer(distanceScore, {'min': 1, 'max': 5, 'palette': rainbow}, 'Distance hazard score', False) 

# Elevation data
srtm = ee.Image("CGIAR/SRTM90_V4")
elevation = srtm.clip(roi1)
Map.addLayer(elevation, {'min': 0, 'max': 100, 'palette': ['green', 'yellow', 'red', 'white']}, 'DEM', False) 

# Elevation score
elevScore = elevation.updateMask(distance.neq(0)) \
    .where(elevation.gt(20), 1) \
    .where(elevation.gt(15).And(elevation.lte(20)), 2) \
    .where(elevation.gt(10).And(elevation.lte(15)), 3) \
    .where(elevation.gt(5).And(elevation.lte(10)), 4) \
    .where(elevation.lte(5), 5)
Map.addLayer(elevScore, {'min': 1, 'max': 5, 'palette': rainbow}, 'Elevation hazard score', False) 

# Create topographic position index
tpi = elevation.subtract(elevation.focalMean(5).reproject('EPSG:4326', None, 30)).rename('TPI')
Map.addLayer(tpi, {'min': -5, 'max': 5, 'palette': ['blue', 'yellow', 'red']}, 'TPI', False) 

# Topo score
topoScore = tpi.updateMask(distance.neq(0)) \
    .where(tpi.gt(0), 1) \
    .where(tpi.gt(-2).And(tpi.lte(0)), 2) \
    .where(tpi.gt(-4).And(tpi.lte(-2)), 3) \
    .where(tpi.gt(-6).And(tpi.lte(-4)), 4) \
    .where(tpi.lte(-8), 5)
Map.addLayer(topoScore, {'min': 1, 'max': 5, 'palette': rainbow}, 'Topographic hazard score', False) 

# Landsat 8
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
landsat8 = l8.filterBounds(roi1).filterDate('2023-01-01', '2023-12-31').map(cloudMask).median().clip(roi1)
Map.addLayer(landsat8, {'min': [0.1, 0.05, 0], 'max': [0.4, 0.3, 0.15], 'bands': ['B5', 'B6', 'B2']}, 'Landsat 8', False) 

# Band map
bandMap = {
    'RED': landsat8.select('B4'),
    'NIR': landsat8.select('B5'),
    'GREEN': landsat8.select('B3'),
}

# NDVI
ndvi = landsat8.expression('(NIR - RED) / (NIR + RED)', bandMap).rename('NDVI')
Map.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI', False) 

# Vegetation score
vegScore = ndvi.updateMask(distance.neq(0)) \
    .where(ndvi.gt(0.8), 1) \
    .where(ndvi.gt(0.6).And(ndvi.lte(0.8)), 2) \
    .where(ndvi.gt(0.4).And(ndvi.lte(0.6)), 3) \
    .where(ndvi.gt(0.2).And(ndvi.lte(0.4)), 4) \
    .where(ndvi.lte(0.2), 5)
Map.addLayer(vegScore, {'min': 1, 'max': 5, 'palette': rainbow}, 'Vegetation hazard score', False) 

# NDWI
ndwi = landsat8.expression('(GREEN - NIR) / (GREEN + NIR)', bandMap).rename('NDWI')
Map.addLayer(ndwi, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'NDWI', False) 

# Wetness score
wetScore = ndwi.updateMask(distance.neq(0)) \
    .where(ndwi.gt(0.6), 5) \
    .where(ndwi.gt(0.2).And(ndwi.lte(0.6)), 4) \
    .where(ndwi.gt(-0.2).And(ndwi.lte(0.2)), 3) \
    .where(ndwi.gt(-0.6).And(ndwi.lte(-0.2)), 2) \
    .where(ndwi.lte(-0.6), 1)
Map.addLayer(wetScore, {'min': 1, 'max': 5, 'palette': rainbow}, 'Wetness hazard score', False) 

# Flood hazard
floodHazard = distanceScore.add(topoScore).add(vegScore).add(wetScore).add(elevScore).rename('Flood_hazard')
Map.addLayer(floodHazard, {'min': 1, 'max': 20, 'palette': rainbow}, 'Flood hazard')

# Flood hazard scored
floodHazardScore = floodHazard.where(floodHazard.gt(15), 5) \
    .where(floodHazard.gt(10).And(floodHazard.lte(15)), 4) \
    .where(floodHazard.gt(5).And(floodHazard.lte(10)), 3) \
    .where(floodHazard.gt(0).And(floodHazard.lte(5)), 2) \
    .where(floodHazard.lte(0), 1)
Map.addLayer(floodHazardScore, {'min': 1, 'max': 5, 'palette': rainbow}, 'Flood hazard score') 

landsat8 = l8.filterBounds(roi1).filterDate('2023-01-01', '2023-12-31').map(cloudMask).median().clip(roi1)
Map


Map(center=[-8.369194293870821, 115.13393742568908], controls=(WidgetControl(options=['position', 'transparent…